In [1]:
!pip install torch
# !pip install transformers datasets rouge_score
import logging
import os
import re
import sys
from dataclasses import dataclass, field
from typing import Optional

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import load_dataset, load_metric
from rouge_score import rouge_scorer
from tqdm import tqdm
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBartTokenizer,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import is_main_process
from datasets import load_dataset

DATA_PATH = "data/"
xsum_train = load_dataset("xsum", split="validation")

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [2]:
!nvidia-smi

/bin/sh: 1: nvidia-smi: not found


In [3]:
"""
Load model
"""

MODEL_NAME = "facebook/bart-base"

config = AutoConfig.from_pretrained(
    MODEL_NAME, cache_dir=None, revision="main", use_auth_token=False,
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, cache_dir=None, use_fast=True, revision="main", use_auth_token=False,
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    MODEL_NAME,
    from_tf=bool(".ckpt" in MODEL_NAME),
    config=None,
    cache_dir=None,
    revision="main",
    use_auth_token=False,
)

In [4]:
# Set decoder_start_token_id
if model.config.decoder_start_token_id is None and isinstance(tokenizer, MBartTokenizer):
    model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.target_lang]
if model.config.decoder_start_token_id is None:
    raise ValueError

column_names = xsum_train.column_names
text_column, summary_column = "document", "summary"

# Temporarily set max_target_length for training.
max_target_length = 128
max_source_length = 1024
padding = False

In [5]:
sep_token = '</s>'
def preprocess_function(examples):

    inputs = examples[text_column]
    targets = examples[summary_column]
    inputs = [inp + sep_token + targets for inp, targets in zip(inputs, targets)]

    # Tokenize Input
    model_inputs = tokenizer(
        inputs, max_length=max_source_length, padding=padding, truncation=True
    )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, max_length=max_target_length, padding=padding, truncation=True
        )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    model_inputs['input_string'] = inputs
    model_inputs['target_string'] = targets
    return model_inputs


train_dataset = xsum_train
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=None,
    remove_columns=column_names,
    load_from_cache_file=False,
)

NameError: name 'xsum_train' is not defined

In [ ]:
from transformers import TrainingArguments

label_pad_token_id = -100
data_collator = DataCollatorForSeq2Seq(tokenizer, label_pad_token_id=label_pad_token_id)

args = TrainingArguments(
    output_dir="exp/bart/results",
    do_train=True,
    do_eval=False,
    evaluation_strategy="steps",
    eval_steps=1000,
    logging_dir="exp/bart/logs",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    eval_accumulation_steps=1,
    warmup_steps=500,
    num_steps=15000
)


# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=None,  # Add Eval DataSet
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=None,
    
)

In [ ]:
train_result = trainer.train()